In [2]:
import capytaine as cpt; print(cpt.__version__)
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from capytaine.io.xarray import separate_complex_values
%matplotlib widget

cpt.set_logging('INFO')
rho_water = 1000.0  # Density of water in kg/m^3

# Generate the mesh of ship; Keep only the immersed part of the mesh
boat_mesh = cpt.load_mesh('2112open.stl')

rotation_center = boat_mesh.center_of_buoyancy # Or specify COG: [x,y,z]


print(f"Mesh loaded. Using rotation center (COB): {rotation_center}")

#lid_mesh = mesh. generate_lid(z=-0.1)
body = cpt.FloatingBody(
        mesh=boat_mesh.immersed_part(),
        dofs=cpt.rigid_body_dofs(rotation_center=rotation_center)
        )
body.show()
print(body.dofs.keys())
# dict_keys(['Surge', 'Sway', 'Heave', 'Roll', 'Pitch', 'Yaw'])
body.center_of_mass          = rotation_center
body.mass                    = body.immersed_part().volume * rho_water
body.inertia_matrix          = body.compute_rigid_body_inertia()



# Hydrostatic calculations
hydrostatics = body.compute_hydrostatics(rho=1000.0)
print("Volume:", hydrostatics["disp_volume"])
print("Center of buoyancy:", hydrostatics["center_of_buoyancy"])
print("Wet surface area:", hydrostatics["wet_surface_area"])
print("Displaced mass:", hydrostatics["disp_mass"])
print("Waterplane center:", hydrostatics["waterplane_center"])
print("Waterplane area:", hydrostatics["waterplane_area"])
print("Metacentric parameters:",
    body.transversal_metacentric_radius,
    body.longitudinal_metacentric_radius,
    body.transversal_metacentric_height,
    body.longitudinal_metacentric_height)

print(hydrostatics["hydrostatic_stiffness"])
# <xarray.DataArray 'hydrostatic_stiffness' (influenced_dof: 6, radiating_dof: 6)>
# [...]
# Coordinates:
#   * influenced_dof  (influenced_dof) <U7 'Surge' 'Sway' ... 'Yaw'
#   * radiating_dof   (radiating_dof) <U7 'Surge' 'Sway' ... 'Yaw'

print(hydrostatics["inertia_matrix"])
# <xarray.DataArray 'inertia_matrix' (influenced_dof: 6, radiating_dof: 6)>
# [...]
# Coordinates:
#   * influenced_dof  (influenced_dof) <U7 'Surge' 'Sway' ... 'Yaw'
#   * radiating_dof   (radiating_dof) <U7 'Surge' 'Sway' ... 'Yaw'

with open('Hydrostatic.txt','w') as fid:
    fid.write('Inertia\n')
    for i in range(6):
        tmp = hydrostatics["inertia_matrix"][i]
        np.savetxt(fid,[tmp],fmt='%.8e')
    fid.write('Hydrostatic stiffness\n')
    for i in range(6):
        tmp = hydrostatics["hydrostatic_stiffness"][i]
        np.savetxt(fid,[tmp],fmt='%.8e')


# Calculations with a test matrix
omega_range = np.arange(2, 15, 0.5) # Example: 1.0, 2.0 rad/s
forward_speeds_range = [0.001,0.1, 0.4, 0.7, 1.0, 1.3] # m/s
wave_amplitude= 0.01
test_matrix = xr.Dataset(coords={
    'omega': omega_range,
    'wave_direction': [np.pi], # Following seas
    'radiating_dof': list(body.dofs.keys()),
    'water_depth': [np.inf],
    'rho': [rho_water],
    'forward_speed': forward_speeds_range
})

dataset = cpt.BEMSolver().fill_dataset(test_matrix, body)
rao = cpt.post_pro.rao(dataset)

separate_complex_values(dataset).to_netcdf("hydrodynamics.nc")
separate_complex_values(rao).to_netcdf("rao.nc")
print("Saved hydrodynamics.nc and rao.nc")


2.2.1
Mesh loaded. Using rotation center (COB): [ 0.00287967 -0.00050748 -0.01680548]


[18:59:18] INFO     The rotation dof Roll has been initialized around the point: FloatingBody(...,                 
                    name="2112open.stl").rotation_center = [ 0.00287967 -0.00050748 -0.01680548]

           INFO     The rotation dof Pitch has been initialized around the point: FloatingBody(...,                
                    name="2112open.stl").rotation_center = [ 0.00287967 -0.00050748 -0.01680548]

           INFO     The rotation dof Yaw has been initialized around the point: FloatingBody(...,                  
                    name="2112open.stl").rotation_center = [ 0.00287967 -0.00050748 -0.01680548]

           INFO     New floating body: FloatingBody(mesh=Mesh(..., name="2112open.stl"), lid_mesh=None,            
                    dofs={"Surge": ..., "Sway": ..., "Heave": ..., "Roll": ..., "Pitch": ..., "Yaw": ...},         
                    name="2112open.stl").

...

[19:49:44] INFO     Solve RadiationProblem(body=FloatingBody(..., name="2112open.stl"), omega=14.500,              
                    water_depth=inf, forward_speed=1.300, wave_direction=3.142, radiating_dof='Roll').

[19:49:46] INFO     Solve RadiationProblem(body=FloatingBody(..., name="2112open.stl"), omega=14.500,              
                    water_depth=inf, forward_speed=1.300, wave_direction=3.142, radiating_dof='Pitch').

[19:49:48] INFO     Solve RadiationProblem(body=FloatingBody(..., name="2112open.stl"), omega=14.500,              
                    water_depth=inf, forward_speed=1.300, wave_direction=3.142, radiating_dof='Yaw').

[19:50:05] INFO     Compute RAO.

TypeError: Cannot interpret 'CategoricalDtype(categories=['Surge', 'Sway', 'Heave', 'Roll', 'Pitch', 'Yaw'], ordered=False, categories_dtype=object)' as a data type

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from capytaine.io.xarray import merge_complex_values
%matplotlib widget

g      = 9.81
weight = 14.16

# reload & restore complex arrays
dataset = merge_complex_values(xr.open_dataset("hydrodynamics.nc"))
rao     = merge_complex_values(xr.open_dataset("rao.nc"))

# rebuild your DOF list
dof_names = list(dataset["radiating_dof"].values)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

g = 9.81  # gravitational acceleration (m/s^2)
weight = 14.16  # constant surge inertia (kg)

# Plot normalized added mass (m_a / weight) vs. encounter frequency
for dof_name in dof_names:
    plt.figure(figsize=(10, 6))
    data_to_plot = dataset['added_mass'].sel(radiating_dof=dof_name, influenced_dof=dof_name)
    omega = dataset['omega'].values  # keep the original omega array intact

    if 'forward_speed' in data_to_plot.dims:
        for U in dataset['forward_speed'].values:
            data_1d = data_to_plot.sel(forward_speed=U).values
            data_1d_norm = data_1d / weight

            k = omega**2 / g
            omega_e = omega - k * U

            # Plot in original data order
            plt.plot(omega_e, data_1d_norm, marker='o', linestyle='-', label=f'U={U:.2f} m/s')

        plt.legend(title='Forward Speed (m/s)')
    else:
        data_1d = data_to_plot.values
        data_1d_norm = data_1d / weight

        k = omega**2 / g
        U = 0.0
        omega_e = omega - k * U

        plt.plot(omega_e, data_1d_norm, marker='o', linestyle='-', label=dof_name)
        plt.legend()

    plt.xlabel(r'Encounter Frequency $\omega_e$ (rad/s)')
    plt.ylabel(f'Added Mass Coefficient $m_a/W$ ({dof_name})')
    plt.title(f'Added Mass Coefficient: {dof_name} vs. Encounter Frequency')
    plt.grid(True)
    plt.tight_layout()

    filename = f'Added_mass_coefficient_{dof_name.replace(" ", "_")}_encounter.png'
    # plt.savefig(filename)
    print(f"Saved plot: {filename}")
    plt.show()

# Plot normalized radiation damping (b / weight) vs. encounter frequency
for dof_name in dof_names:
    plt.figure(figsize=(10, 6))
    data_to_plot = dataset['radiation_damping'].sel(radiating_dof=dof_name, influenced_dof=dof_name)
    omega = dataset['omega'].values

    if 'forward_speed' in data_to_plot.dims:
        for U in dataset['forward_speed'].values:
            data_1d = data_to_plot.sel(forward_speed=U).values
            data_1d_norm = data_1d / weight

            k = omega**2 / g
            omega_e = omega - k * U

            plt.plot(omega_e, data_1d_norm, marker='s', linestyle='--', label=f'U={U:.2f} m/s')

        plt.legend(title='Forward Speed (m/s)')
    else:
        data_1d = data_to_plot.values
        data_1d_norm = data_1d / weight

        k = omega**2 / g
        U = 0.0
        omega_e = omega - k * U

        plt.plot(omega_e, data_1d_norm, marker='s', linestyle='--', label=dof_name)
        plt.legend()

    plt.xlabel(r'Encounter Frequency $\omega_e$ (rad/s)')
    plt.ylabel(f'Radiation Damping Coefficient $b/W$ ({dof_name})')
    plt.title(f'Radiation Damping Coefficient: {dof_name} vs. Encounter Frequency')
    plt.grid(True)
    plt.tight_layout()

    filename = f'Radiation_damping_coefficient_{dof_name.replace(" ", "_")}_encounter.png'
    # plt.savefig(filename)
    print(f"Saved plot: {filename}")
    plt.show()
